In [83]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json
from pyspark.sql import SparkSession
import os
import re 

#folium
import geopandas as gpd
import re
import folium
from shapely.geometry import Point

In [306]:
#import postcode shape file
postcode_df = gpd.read_file("../data/raw/POSTCODE/POSTCODE_POLYGON.shp")

# add postcode column based on geometry column of given dataframe
def add_postcode_column_from_geometry(df):
    lst = []
    for i in range(0, len(df)):
        point = Point(df['geometry'].values.x[i], df['geometry'].values.y[i])
        for j in range(0, len(postcode_df)):
            current_postcode = postcode_df.loc[j,'POSTCODE']
            polygon = postcode_df.loc[j,'geometry']
            if point.within(polygon):
                lst.append(current_postcode)
                break
    return lst

In [193]:
output_dir = '../data/raw/'

In [194]:
'''


property and elector count by postcode


'''

#url: https://discover.data.vic.gov.au/dataset/victorian-electors-by-locality-postcode-and-electorates 


columns = ['Locality Name','Post_x000D_\nCode','Property_x000D_\nCount','Elector_x000D_\nCount']

LocalityFinder_postcode = pd.read_excel('../data/raw/LocalityFinder_postcode.xlsx',
                                        sheet_name= 'Place_Names_Electronic',
                                        header=2)

LocalityFinder_postcode= LocalityFinder_postcode[columns].rename({'Post_x000D_\nCode': 'postcode',
                                                             'Locality Name':'suburb_name',
                                                             'Property_x000D_\nCount': 'property_count',
                                                            'Elector_x000D_\nCount':'elector_count'}, axis='columns')
property_and_elector = LocalityFinder_postcode.groupby('postcode').sum()

#output
filename = 'property_and_elector_by_postcode.csv'
output_dir_full = f'{output_dir}{filename}'
property_and_elector.to_csv(output_dir_full)

In [195]:
"""
Download PTV zip file
1. open url: https://discover.data.vic.gov.au/dataset/ptv-metro-train-stations
2. select PTV shape file, add to cart and check out with email
3. open url in email recieved, download zip file
4. find folder with name 'PTV'
5. move folder under raw data folder


"""

"\nDownload PTV zip file\n1. open url: https://discover.data.vic.gov.au/dataset/ptv-metro-train-stations\n2. select PTV shape file, add to cart and check out with email\n3. open url in email recieved, download zip file\n4. find folder with name 'PTV'\n5. move folder under raw data folder\n\n\n"

In [266]:
#PTV
ptv_df = gpd.read_file("../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp")
selected_columns = ['STOP_ID','STOP_NAME','geometry',
                   'LATITUDE','LONGITUDE','TICKETZONE']
ptv_df = ptv_df[selected_columns]

#add postcode column
postcode_lst = add_postcode_column_from_geometry(ptv_df)
ptv_df['postcode'] = postcode_lst

In [267]:
ptv_df

,STOP_ID,STOP_NAME,geometry,LATITUDE,LONGITUDE,TICKETZONE,postcode
0,19970,Royal Park Railway Station (Parkville),POINT (144.95231 -37.78118),-37.781193,144.952301,1,3052
1,19971,Flemington Bridge Railway Station (North Melbo...,POINT (144.93933 -37.78813),-37.788140,144.939323,1,3051
2,19972,Macaulay Railway Station (North Melbourne),POINT (144.93617 -37.79425),-37.794267,144.936166,1,3051
3,19973,North Melbourne Railway Station (West Melbourne),POINT (144.94258 -37.80741),-37.807419,144.942570,1,3003
4,19974,Clifton Hill Railway Station (Clifton Hill),POINT (144.99542 -37.78864),-37.788657,144.995417,1,3068
...,...,...,...,...,...,...,...
215,19965,Coburg Railway Station (Coburg),POINT (144.96334 -37.74233),-37.742345,144.963336,1,3058
216,19966,Moreland Railway Station (Coburg),POINT (144.96183 -37.75447),-37.754485,144.961823,1,3058
217,19967,Anstey Railway Station (Brunswick),POINT (144.96069 -37.76123),-37.761242,144.960684,1,3056
218,19968,Brunswick Railway Station (Brunswick),POINT (144.95959 -37.76771),-37.767721,144.959587,1,3056


In [268]:
#output
filename = 'train_staiton.csv'
output_dir_full = f'{output_dir}{filename}'
ptv_df.to_csv(output_dir_full)

In [198]:
"""
Download FOI zip file
1. open url:https://datashare.maps.vic.gov.au/search?md=019d7631-1234-5112-9f21-8f7346647b61
2. add to cart and check out with email
3. open url in email recieved, download zip file
4. find folder with name 'VMFEAT'
5. move folder under raw data folder


"""

"\nDownload FOI zip file\n1. open url:https://datashare.maps.vic.gov.au/search?md=019d7631-1234-5112-9f21-8f7346647b61\n2. add to cart and check out with email\n3. open url in email recieved, download zip file\n4. find folder with name 'VMFEAT'\n5. move folder under raw data folder\n\n\n"

In [199]:
# sf stands for shape file
sf = gpd.read_file("../data/raw/VMFEAT/FOI_POINT.shp")
VIC_FOI = sf.loc[sf['STATE']=='VIC']

In [ ]:
'''


hospital


'''
comunity_sector = ['communication service','NAME','geometry']
hospital = VIC_FOI.loc[VIC_FOI['FTYPE'] == 'hospital']
hospital_columns = ['FEATSUBTYP','NAME','geometry']

hospital_df = hospital[hospital_columns]
hospital_df = hospital_df.dropna(subset=['FEATSUBTYP','NAME','geometry'])
hospital_df['geometry'] = hospital_df['geometry'].centroid


#add postcode column
postcode_lst = add_postcode_column_from_geometry(hospital_df)
hospital_df['postcode'] = postcode_lst

#output
filename = 'hospital.csv'
output_dir_full = f'{output_dir}{filename}'
hospital_df.to_csv(output_dir_full)


In [253]:
'''

emergency services: police station, amubulance


'''
import geopandas as gpd
#filter out important emergency sevice
target_emergency_service= ['police station',
                           'ambulance station',
                           'fire station']
selected_columns = ['NAME_LABEL','FEATSUBTYP','geometry']

emergency_service_df = VIC_FOI.loc[VIC_FOI['FTYPE'] == 'emergency facility'][selected_columns]
emergency_service_df = emergency_service_df.loc[emergency_service_df['FEATSUBTYP']\
                                                .isin(target_emergency_service)]
emergency_service_df = emergency_service_df.dropna(subset=['NAME_LABEL','geometry'])
emergency_service_df['geometry'] = emergency_service_df['geometry'].centroid


#add postcode column
postcode_lst = add_postcode_column_from_geometry(emergency_service_df)
emergency_service_df['postcode'] = postcode_lst


/var/folders/7v/v1gh4ksn641cv86zx_w_yd840000gp/T/ipykernel_52440/3599235357.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  emergency_service_df['geometry'] = emergency_service_df['geometry'].centroid


In [269]:
emergency_service_df

,NAME_LABEL,FEATSUBTYP,geometry,postcode
2244,Alexandra Police Station,police station,POINT (145.70778 -37.18862),3714
2943,Anglesea Police Station,police station,POINT (144.19223 -38.40339),3230
2996,Kyneton Ambulance Station,ambulance station,POINT (144.46919 -37.25439),3444
2998,Landsborough Police Station,police station,POINT (143.12854 -37.00363),3384
2999,Lexton Police Station,police station,POINT (143.51382 -37.27525),3352
...,...,...,...,...
42616,Clayton Fire Station,fire station,POINT (145.11665 -37.93900),3169
42636,Croydon Fire Station,fire station,POINT (145.28942 -37.79940),3136
42639,Deer Park Fire Station,fire station,POINT (144.77839 -37.77039),3023
42646,Eastern Hill Fire Station,fire station,POINT (144.97545 -37.80889),3002


In [255]:
#output
filename = 'emergency_service.csv'
output_dir_full = f'{output_dir}{filename}'
emergency_service_df.to_csv(output_dir_full)

In [311]:
'''


public service: 
swimming pool,libary museum,art gallery


'''
service_type= ['sport facility',
               'cultural centre',
               'community space']

public_service = VIC_FOI.loc[VIC_FOI['FTYPE']\
                             .isin(service_type)]
target_service = ['swimming pool',
                  'library',
                  'museum',
                  'art gallery',
                  'aquarium',
                  'observatory']
selected_columns = ['FTYPE','FEATSUBTYP','NAME','geometry'] 
public_service = public_service.loc[public_service['FEATSUBTYP']\
                             .isin(target_service)][selected_columns]
public_service_df = public_service.dropna(subset=['NAME','geometry'])
public_service_df['geometry'] = public_service['geometry'].centroid

#add postcode column
postcode_lst = add_postcode_column_from_geometry(public_service_df)
public_service_df['postcode'] = postcode_lst


/var/folders/7v/v1gh4ksn641cv86zx_w_yd840000gp/T/ipykernel_52440/3796321038.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  public_service_df['geometry'] = public_service['geometry'].centroid
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [312]:
public_service_df

,FTYPE,FEATSUBTYP,NAME,geometry,postcode
1515,sport facility,swimming pool,STRATHMERTON OUTDOOR POOL,POINT (145.47805 -35.92389),3641
2207,sport facility,swimming pool,AQUALINK BOX HILL,POINT (145.11800 -37.82562),3128
2277,cultural centre,library,ENDEAVOUR HILLS LIBRARY,POINT (145.25989 -37.97775),3802
2427,cultural centre,library,NEWPORT LIBRARY,POINT (144.88182 -37.84274),3015
2572,cultural centre,library,LANCEFIELD LIBRARY,POINT (144.73552 -37.27692),3435
...,...,...,...,...,...
42815,cultural centre,museum,STATE COAL MINE MUSEUM,POINT (145.59733 -38.62364),3995
42903,cultural centre,art gallery,ABBOTSFORD CONVENT,POINT (145.00453 -37.80296),3067
42943,sport facility,swimming pool,HIDDEN VALLEY SWIMMING POOL,POINT (144.99411 -37.39462),3756
43409,cultural centre,library,LEOPOLD LIBRARY,POINT (144.46432 -38.18595),3224


In [314]:
#output
filename = 'public_service.csv'
output_dir_full = f'{output_dir}{filename}'
public_service_df.to_csv(output_dir_full)



In [271]:
'''


care facility (child,disability,aged)



'''

selected_columns = ['NAME_LABEL','FEATSUBTYP','geometry']
care_facility_df = VIC_FOI.loc[VIC_FOI['FTYPE'] =='care facility'][selected_columns]
care_facility_df['geometry'] = care_facility_df['geometry'].centroid


#add postcode column
postcode_lst = add_postcode_column_from_geometry(care_facility_df)
care_facility_df['postcode'] = postcode_lst

#output
filename = 'care_facility.csv'
output_dir_full = f'{output_dir}{filename}'
care_facility_df.to_csv(output_dir_full)




/var/folders/7v/v1gh4ksn641cv86zx_w_yd840000gp/T/ipykernel_52440/1930701637.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  care_facility_df['geometry'] = care_facility_df['geometry'].centroid


In [315]:
care_facility_df

,NAME_LABEL,FEATSUBTYP,geometry,postcode
2992,Infant Welfare Centre,disability support centre,POINT (147.08039 -37.96556),3862
10296,Villa Maria Society,disability support centre,POINT (147.06835 -38.10475),3850
10568,Kyeema Support Services,disability support centre,POINT (141.58265 -38.34137),3305
10678,Cooinda Day Centre,disability support centre,POINT (141.66050 -36.32953),3418
11639,Aurrum Reservoir,aged care,POINT (145.01799 -37.70365),3073
...,...,...,...,...
48124,Coburg Childrens Centre Inc.,child care,POINT (144.97040 -37.74139),3058
48131,Childs Play Torquay,child care,POINT (144.34140 -38.30323),3228
48232,Eastwood Early Learning Centre And Kindergarten,child care,POINT (147.64152 -37.80822),3875
48245,Parkmore Communityosh,child care,POINT (145.17646 -37.84408),3131
